In [ ]:
!pip install azureml-sdk

In [64]:
import os
import json
import requests

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core import Experiment,ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import AmlCompute, ComputeTarget

In [65]:
# laoding the configuration file - standard way - use .env file and load_dotenv from python-dotenv module
config_file_path = "config.json"

# Read JSON data into a dictionary
with open(config_file_path, 'r') as file:
    data = json.load(file)

subscription_id = data["subscription_id"]
resource_group = data["resource_group"]  
workspace_name = data["workspace_name"]
region = data["region"]
compute_target = data["compute_target"]

In [66]:
print(resource_group)
print(workspace_name)
print(region)
print(compute_target)

PracticeMachineLearning
PracticeMachineLearning
centralus
PracticeML


In [67]:
try:
    ws=Workspace.from_config()
    print('Workspace is already exist')
except:
    ws=Workspace.create(workspace_name, 
                    resource_group=resource_group,
                    create_resource_group=True,
                    subscription_id=subscription_id,
                    region = region)
    ws.write_config('.azureml')

Workspace is already exist


In [68]:
Workspace.from_config()

Workspace.create(name='PracticeMachineLearning', subscription_id='205478b7-1f8c-4ae8-be05-317e7dc7d199', resource_group='PracticeMachineLearning')

In [69]:
# Create Compute Target
try:
    aml_compute = AmlCompute(ws, compute_target)
    print("This Compute Target already exist.")
except ComputeTargetException:
    print("creating new compute target :",compute_target)
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4,
                                               idle_seconds_before_scaledown=3000)    
    aml_compute = ComputeTarget.create(ws, compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached now")

This Compute Target already exist.
Azure Machine Learning Compute attached now


In [70]:
# Specify the path to your  model file
model_path = 'RandomForestRegressor.pkl'

In [71]:
model_name='Wine_Quality_RandForestRegressor'

In [72]:
registered_model = Model.register(model_path=model_path, model_name=model_name, workspace=ws)

Registering model Wine_Quality_RandForestRegressor


In [73]:
conda_env = Environment('AzureDeploy-Wine')
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [74]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='Score.py', environment=conda_env)

In [75]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=5)

In [77]:
service = Model.deploy(workspace=ws,
                       name='wine-quality-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

C:\Users\Chirayu Baliyan\AppData\Local\Temp\ipykernel_34596\1088263956.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-03-13 22:23:26+05:30 Creating Container Registry if not exists.
2024-03-13 22:23:26+05:30 Registering the environment.
2024-03-13 22:23:28+05:30 Use the existing image.
2024-03-13 22:23:28+05:30 Generating deployment configuration.
2024-03-13 22:23:28+05:30 Submitting deployment to compute.
2024-03-13 22:23:33+05:30 Checking the status of deployment wine-quality-service..
2024-03-13 22:25:16+05:30 Checking the status of inference endpoint wine-quality-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [78]:
scoring_uri = service.scoring_uri

In [79]:
scoring_uri

'http://7a98e220-d48e-436d-92df-e65ed0fbb0b6.centralus.azurecontainer.io/score'